### Задание по программированию: Метрики качества классификации

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve

#### 1. Загрузите файл classification.csv. В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка predicted)

In [2]:
data = pd.read_csv('./classification.csv')
data.head()

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1


#### 2. Заполните таблицу ошибок классификации. Для этого подсчитайте величины TP, FP, FN и TN согласно их определениям.

In [3]:
charac = {'tp':(1, 1), 'fp':(0, 1), 'fn':(1, 0), 'tn':(0, 0)}
classf_table = {n: len(data[(data['true'] == ch[0]) & (data['pred'] == ch[1])]) for n, ch in charac.iteritems()}
'{tp} {fp} {fn} {tn}'.format(**classf_table)

'43 34 59 64'

#### 3. Посчитайте основные метрики качества классификатора:
- Accuracy (доля верно угаданных) — sklearn.metrics.accuracy
- Precision (точность) — sklearn.metrics.accuracy.precision_score
- Recall (полнота) — sklearn.metrics.recall_score
- F-мера — sklearn.metrics.f1_score

In [4]:
metrics = {'accuracy':accuracy_score, 'precision':precision_score, 'recall':recall_score, 'f1':f1_score}
{n: f(data['pred'], data['true']) for n, f in metrics.iteritems()}

{'accuracy': 0.535,
 'f1': 0.48044692737430167,
 'precision': 0.4215686274509804,
 'recall': 0.5584415584415584}

#### 4. Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:
- для логистической регрессии — вероятность положительного класса (колонка score_logreg),
- для SVM — отступ от разделяющей поверхности (колонка score_svm),
- для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),
- для решающего дерева — доля положительных объектов в листе (колонка score_tree).

In [5]:
scores_data = pd.read_csv('./scores.csv')
scores_data.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


#### 5. Посчитайте площадь под ROC-кривой для каждого классификатора. Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца с ответами этого классификатора)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [6]:
roc_scores = {}
for classifier in scores_data.columns[1:]:
    roc_scores[classifier] = roc_auc_score(scores_data['true'], scores_data[classifier])

pd.Series(roc_scores).sort_values(ascending=False).head(1)

score_logreg    0.719188
dtype: float64

#### 6. Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% (укажите название столбца с ответами этого классификатора)? Какое значение точности при этом получается?

#### Чтобы получить ответ на этот вопрос, найдите все точки precisionrecall-кривой с помощью функции sklearn.metrics.precision_recall_curve.
#### Она возвращает три массива: precision, recall, thresholds. В них записаны точность и полнота при определенных порогах, указанных в массиве thresholds. Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.

In [7]:
precision_scores = {}

for classifier in scores_data.columns[1:]:
    precision_curve = precision_recall_curve(scores_data['true'], scores_data[classifier])
    precision_curve_df = pd.DataFrame({'precision': precision_curve[0], 'recall': precision_curve[1]})
    precision_scores[classifier] = precision_curve_df[precision_curve_df['recall'] > 0.7]['precision'].max()

pd.Series(precision_scores).sort_values(ascending=False).head(1)

score_tree    0.651786
dtype: float64